In [16]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout ,LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, r2_score

In [17]:

# Step 2: Reading Data
df1 = pd.read_csv('422 datasets/madrid_2001.csv')
df2 = pd.read_csv('422 datasets/madrid_2002.csv')
df3 = pd.read_csv('422 datasets/madrid_2003.csv')
df4 = pd.read_csv('422 datasets/madrid_2004.csv')
df5 = pd.read_csv('422 datasets/madrid_2005.csv')
df6 = pd.read_csv('422 datasets/madrid_2006.csv')
df7 = pd.read_csv('422 datasets/madrid_2007.csv')
df8 = pd.read_csv('422 datasets/madrid_2008.csv')
df9 = pd.read_csv('422 datasets/madrid_2009.csv')
df10 = pd.read_csv('422 datasets/madrid_2010.csv')
df11 = pd.read_csv('422 datasets/madrid_2011.csv')
df12 = pd.read_csv('422 datasets/madrid_2012.csv')
df13 = pd.read_csv('422 datasets/madrid_2013.csv')
df14 = pd.read_csv('422 datasets/madrid_2014.csv')
df15 = pd.read_csv('422 datasets/madrid_2015.csv')
df16 = pd.read_csv('422 datasets/madrid_2016.csv')
df17 = pd.read_csv('422 datasets/madrid_2017.csv')
df18 = pd.read_csv('422 datasets/madrid_2018.csv')


# Concatenate all dataframes
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18], axis=0)


# Reset the index of the concatenated dataframe
df = df.reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)


df.head()


,date,BEN,CO,EBE,MXY,NMHC,NO_2,NOx,OXY,O_3,PM10,PXY,SO_2,TCH,TOL,station,PM25,NO,CH4
0,2001-08-01 01:00:00,NaN,0.37,NaN,NaN,NaN,58.400002,87.150002,NaN,34.529999,105.000000,NaN,6.34,NaN,NaN,28079001,NaN,NaN,NaN
1,2001-08-01 01:00:00,1.5,0.34,1.49,4.1,0.07,56.250000,75.169998,2.11,42.160000,100.599998,1.73,8.11,1.24,10.82,28079035,NaN,NaN,NaN
2,2001-08-01 01:00:00,NaN,0.28,NaN,NaN,NaN,50.660000,61.380001,NaN,46.310001,100.099998,NaN,7.85,NaN,NaN,28079003,NaN,NaN,NaN
3,2001-08-01 01:00:00,NaN,0.47,NaN,NaN,NaN,69.790001,73.449997,NaN,40.650002,69.779999,NaN,6.46,NaN,NaN,28079004,NaN,NaN,NaN
4,2001-08-01 01:00:00,NaN,0.39,NaN,NaN,NaN,22.830000,24.799999,NaN,66.309998,75.180000,NaN,8.80,NaN,NaN,28079039,NaN,NaN,NaN


In [18]:
df=df.drop(['EBE', 'PM10','station'], axis = 1)
df.head()

,date,BEN,CO,MXY,NMHC,NO_2,NOx,OXY,O_3,PXY,SO_2,TCH,TOL,PM25,NO,CH4
0,2001-08-01 01:00:00,NaN,0.37,NaN,NaN,58.400002,87.150002,NaN,34.529999,NaN,6.34,NaN,NaN,NaN,NaN,NaN
1,2001-08-01 01:00:00,1.5,0.34,4.1,0.07,56.250000,75.169998,2.11,42.160000,1.73,8.11,1.24,10.82,NaN,NaN,NaN
2,2001-08-01 01:00:00,NaN,0.28,NaN,NaN,50.660000,61.380001,NaN,46.310001,NaN,7.85,NaN,NaN,NaN,NaN,NaN
3,2001-08-01 01:00:00,NaN,0.47,NaN,NaN,69.790001,73.449997,NaN,40.650002,NaN,6.46,NaN,NaN,NaN,NaN,NaN
4,2001-08-01 01:00:00,NaN,0.39,NaN,NaN,22.830000,24.799999,NaN,66.309998,NaN,8.80,NaN,NaN,NaN,NaN,NaN


In [19]:
df= df.dropna(subset=['O_3']) #i deleted all null rows to check the result


print('New Shape after dropping null rows of  value:')
df.shape

New Shape after dropping null rows of  value:


(2991732, 16)

In [21]:
df['CO'].fillna(df['CO'].mean(), inplace = True)
df['NO_2'].fillna(df['NO_2'].mean(), inplace = True)
df['SO_2'].fillna(df['SO_2'].mean(), inplace = True)

df['BEN'].fillna(df['BEN'].mean(), inplace = True)
df['MXY'].fillna(df['MXY'].mean(), inplace = True)
df['NMHC'].fillna(df['NMHC'].mean(), inplace = True)
df['NOx'].fillna(df['NOx'].mean(), inplace = True)


df['OXY'].fillna(df['OXY'].mean(), inplace = True)
df['PXY'].fillna(df['PXY'].mean(), inplace = True)
df['SO_2'].fillna(df['SO_2'].mean(), inplace = True)
df['TCH'].fillna(df['TCH'].mean(), inplace = True)


df['TOL'].fillna(df['TOL'].mean(), inplace = True)
df['PM25'].fillna(df['PM25'].mean(), inplace = True)
df['NO'].fillna(df['NO'].mean(), inplace = True)
df['CH4'].fillna(df['CH4'].mean(), inplace = True)

In [22]:
#Outlier treatment
def wisker(col):
  q1,q3=np.percentile(col,[25,75])
  iqr=q3-q1
  lower_bound=q1-(1.5*iqr)
  upper_bound=q3+(1.5*iqr)
  return lower_bound,upper_bound
df.columns

for i in ['BEN', 'CO', 'MXY', 'NMHC','SO_2', 'PM25']:
  lw,uk=wisker(df[i])
  df[i]=np.where(df[i]>uk,uk,df[i])
  df[i]=np.where(df[i]<lw,lw,df[i])

In [ ]:
target_column = 'O_3'  # Update this to the actual target column name

# Convert datetime columns to numerical values
for col in df.select_dtypes(include=[np.datetime64]):
    df[col] = (df[col] - df[col].min()).dt.total_seconds()

# Create a copy of the initial dataframe for reference
df_init = df.copy()

# Separate features (X) and target (y)
y = df[target_column].values  # Convert to numpy array
X = df.drop(columns=[target_column]).values  # Drop the target column

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False) 

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input data for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # Adding time dimension
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the RNN model with LSTM layers
model = Sequential([
    LSTM(64, return_sequences=True, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32, return_sequences=False, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  # Regression output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae','r2_score'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
y_pred = model.predict(X_test).flatten()  # Flatten predictions for evaluation

# Calculate R² Score
r2 = r2_score(y_test, y_pred)
mae = np.mean(np.abs(y_pred - y_test))

print(f'Mean Absolute Error: {mae:.2f}')
print(f'R² Score: {r2:.2f}')

/home/t2420363/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:1037: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask
/home/t2420363/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
37397/37397 ━━━━━━━━━━━━━━━━━━━━ 56s 1ms/step - loss: 464.6708 - mae: 15.5352 - r2_score: 0.4398 - val_loss: 900.1981 - val_mae: 21.6059 - val_r2_score: 0.2261
Epoch 2/100
37397/37397 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - loss: 399.5900 - mae: 14.5671 - r2_score: 0.5201 - val_loss: 1100.2086 - val_mae: 24.1268 - val_r2_score: 0.0542
Epoch 3/100
37397/37397 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - loss: 391.4576 - mae: 14.4059 - r2_score: 0.5286 - val_loss: 1119.5880 - val_mae: 24.3857 - val_r2_score: 0.0375
Epoch 4/100
37397/37397 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - loss: 387.9409 - mae: 14.3297 - r2_score: 0.5333 - val_loss: 1061.9136 - val_mae: 23.5906 - val_r2_score: 0.0871
Epoch 5/100
37397/37397 ━━━━━━━━━━━━━━━━━━━━ 56s 2ms/step - loss: 385.3145 - mae: 14.2780 - r2_score: 0.5360 - val_loss: 957.3298 - val_mae: 22.3250 - val_r2_score: 0.1770
Epoch 6/100
37397/37397 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - loss: 384.1143 - mae: 14.2580 - r2_score: 0.5385 - val_loss: 950.2043 - val_m

[]
